You will need to install (via pip3): gensim, nltk.  
You will also need to run (with python3 in terminal)  
`>>>import nltk`  
`>>>nltk.download('stopwords')`  
`>>>nltk.download('wordnet')`  
`>>>nltk.download('omw-1.4')`

In [1]:
import pandas as pd
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from collections import Counter
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
import torch
import pickle

with open('stock_data.csv', encoding='utf8') as csvfile:
    df = pd.read_csv(csvfile, delimiter=',')

df.dropna(axis=0, how='any', inplace=True)                         # Excludes null-containing rows
num_positive = df['Sentiment'].value_counts()[1]
num_negative = df['Sentiment'].value_counts()[-1]
print(df['Sentiment'].value_counts())

 1    3685
-1    2106
Name: Sentiment, dtype: int64


In [2]:
# Hyperparameters
word_frequency_requirement = 0.0013*(df['Sentiment'].size) # the number of times a word has to appear to be given
# it's own encoding. All words under this limit are encoded as the same 'unknown' word.
sg = 0
vector_size = 10000

In [3]:
# Regex removal of various undesirable parts of a tweet
def clean_tweet(tweet):
  tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet) # Twitter handle removal
  tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet) # URL removal
  tweet = re.sub(r"[']", "", tweet) # Apostrophe removal
  tweet = re.sub(r"[^a-zA-Z.!?]", ' ', tweet) # Remove symbols that are not alphabetic or sentence endings
  tweet = re.sub(r"([^a-zA-Z])", r" \1 ", tweet) # Places spaces around sentence endings,
  # so they are encoded as their own words, rather than being lumped in with other words.
  tweet = re.sub(r" +", ' ', tweet) # Excess whitespace removal
  tweet = tweet.lower() # Send tweet to lowercase
  return tweet

In [4]:
# Prepare word lemmatizer and stopwords list for sanitisation
lemmatizer = WordNetLemmatizer()
stops = set(stopwords.words('english'))

def sanitise(tweet):
    tweet = clean_tweet(tweet)
    tweet = filter(lambda w: w not in stops, tweet.strip().split()) # Remove stopwords
    return list(map(lemmatizer.lemmatize, tweet)) # Lemmatize words.

In [5]:
sentences = list(df['Text'].map(sanitise))
model= Word2Vec(sentences, min_count=1, vector_size=vector_size, sg=sg)      # default size=100, sg=0 CBOW, min_count=5

In [6]:
wv = model.wv # get word vector

In [7]:
encoded_df = pd.DataFrame([
    [[wv[word] for word in sentence] for sentence in sentences],
    df['Sentiment'].map(lambda x: torch.tensor([1]) if (x==1) else torch.tensor([0]))
]).T

max_tweet_length = max(len(x) for x in encoded_df[0])

encoded_df[0] = encoded_df[0].map( lambda x: x + [[0]*vector_size] * (max_tweet_length - len(x)) )

encoded_df[0] = encoded_df[0].map(lambda x: torch.FloatTensor(x))

with open("encoded_dataframe", "wb") as encoded_dataframe:
    pickle.dump(encoded_df, encoded_dataframe)

C:\Users\Redmond\AppData\Local\Temp\ipykernel_11856\3665440953.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:204.)
  encoded_df[0] = encoded_df[0].map(lambda x: torch.FloatTensor(x))
